# Import libraries

In [56]:
import sqlite3
import requests
import time
import os
from dotenv import load_dotenv

# Database

In [57]:
# Create and/or connect to database
conn = sqlite3.connect("seq_data.db")
cur = conn.cursor()

In [58]:
# Create a company dimension table
cur.execute("""
CREATE TABLE IF NOT EXISTS dim_company (
    cik INTEGER PRIMARY KEY,
    cik_padded TEXT NOT NULL,
    ticker TEXT,
    company_name TEXT NOT NULL,
    mgt_insert_timestamp TEXT DEFAULT (datetime('now')),
    mgt_update_timestamp TEXT DEFAULT (datetime('now'))
);
""")

# Create trigger where we update timestamps on first insert
cur.execute("""
CREATE TRIGGER IF NOT EXISTS trg_dim_company_insert
AFTER INSERT ON dim_company
BEGIN
    UPDATE dim_company
    SET mgt_insert_timestamp = datetime('now'),
        mgt_update_timestamp = datetime('now')
    WHERE cik = NEW.cik;
END;
""")

# Create trigger where we change the update timestamp on row update
cur.execute("""
CREATE TRIGGER IF NOT EXISTS trg_dim_company_update
AFTER UPDATE ON dim_company
BEGIN
    UPDATE dim_company
    SET mgt_update_timestamp = datetime('now')
    WHERE cik = NEW.cik;
END;
""")

# Get data

In [59]:
# Get our user agent header data from .env
# Has format "name e-mail address"
load_dotenv()
user_agent = os.getenv('user_agent')

In [60]:
# Get all the different companies listed at the SEC
headers = {"User-Agent": user_agent}
url = "https://www.sec.gov/files/company_tickers.json"
res = requests.get(url, headers=headers)
companies = res.json()

In [61]:
# Make an array with the company data and insert it into our dimension table
# The cik number is a unique identifier for the company
# A padded cik number with 0s to length 10 is required for certain endpoints
# The ticker is a short string identifier for the company
# Company name is the full name of the company
company_data = []
for _, c in companies.items():
	company_data.append((
		c['cik_str'],
		str(c['cik_str']).zfill(10),
		c.get('ticker'),
		c['title']
	))

cur.executemany("""
INSERT INTO dim_company (cik, cik_padded, ticker, company_name)
VALUES (?, ?, ?, ?)
ON CONFLICT(cik) DO UPDATE SET
    cik_padded = excluded.cik_padded,
    ticker = excluded.ticker,
    company_name = excluded.company_name
WHERE cik_padded != excluded.cik_padded
   OR ticker != excluded.ticker
   OR company_name != excluded.company_name;
""", company_data)

conn.commit()